In [1]:
from glob import glob
import json
import re
import yaml
import os
import shutil
import time
os.chdir('../')

In [2]:
with open("config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)

In [3]:
queues_cfg = cfg['QUEUES']

In [4]:
tweets_queue_path = queues_cfg['new_tweets']
processed_tweets_path = queues_cfg['processed_tweets']
sa_queue_path = queues_cfg['sentiment_tasks']
geo_queue_path = queues_cfg['geo_tasks']

In [5]:
#for folder in [tweets_folder, sentiment_folder, geo_folder, 'tweet_no_coordinates']:
for folder in [tweets_queue_path, processed_tweets_path, ]:
    if not os.path.exists(folder):
        os.makedirs(folder)

In [6]:
unprocessed_tweets = glob('{}/*.json'.format(tweets_queue_path))
len(unprocessed_tweets)

56139

# Process Downloaded Tweets Queue

In [7]:
def create_geoanalyser_task(tweet_id, coordinates):
    filename = '{}/{}.task.txt'.format(geo_queue_path, tweet_id)
    with open(filename, 'w') as fp:
        fp.write(json.dumps(coordinates))
        fp.close()

def send_to_geoanalyser(tweet_id, tweet_json):
    try:
        if tweet_json['coordinates'] is not None:
            coordinates = tweet_json['coordinates']
        elif tweet_json['place'] is not None:
            coordinates = tweet_json['place']['bounding_box']
        else:
            print('Tweet {} doesn\'t contain coordinates'.format(tweet_id))
            print(path)
        create_geoanalyser_task(tweet_id, coordinates=coordinates)
    except Exception as e:
        print('Tweet {} wasn\'t sent to geoanalyser due to error. {}'.format(tweet_id, e))
        #raise e

In [8]:
def create_sentiment_task(tweet_id, text):
    filename = '{}/{}.task.txt'.format(sa_queue_path, tweet_id)
    with open(filename, 'w') as fp:
        fp.write(text)
        fp.close()
        
def send_to_sentiment_analysis(tweet_id, tweet_json):
    try:
        if tweet_json['text'] is not None:
            text = tweet_json['text']
        else:
            print('Tweet {} doesn\'t contain text'.format(tweet_id))
            print(path)
        create_sentiment_task(tweet_id, text)
    except Exception as e:
        print('Tweet {} wasn\'t sent to sentiment analyser due to error. {}'.format(tweet_id, e))

In [9]:
def save_to_couch_db(path):
    filename = path.split('/')[-1]
    new_path = '{}/{}'.format(processed_tweets_path, filename)
    shutil.move(path, new_path)

In [10]:
!cp $processed_tweets_path/* $tweets_queue_path

In [ ]:
#for path in bitcoin_pathes:
i = 1
while True:
    unprocessed_tweets = glob('{}/*.json'.format(tweets_queue_path))
    for path in unprocessed_tweets:
        with open(path, 'r') as fp:
            tweet_json = json.load(fp)
            tweet_id = tweet_json['id_str']
            for analyser_func in (send_to_geoanalyser, send_to_sentiment_analysis):
                analyser_func(tweet_id, tweet_json)
            save_to_couch_db(path)
    
    print('Iteration: {}\tFiles processed: {}'.format(i, len(unprocessed_tweets)))
    i+=1
    time.sleep(20)
    
        

Iteration: 1	File processed: 5
Iteration: 2	File processed: 4
Iteration: 3	File processed: 5
Iteration: 4	File processed: 11
Iteration: 5	File processed: 6
Iteration: 6	File processed: 11


## test

In [15]:
with open('../shared_folder/tweets/988986510979497985.json', 'r') as fp:
    data = json.load(fp)   
#data

# Analyse tweets contents

In [8]:
hashtags = ['bitcoin', 'blockchain', 'btc', 'cryptocurrency', 'market\svalue',
            'crypto','ethereum', 'fintech', 'coin', 'doge', 'ethereum', 'ripple', 'litecoin', 
            'cardano', 'monero', 'TRON', 'zcash', 'jaxx', 'copay', 'bitpay', 'exodus', 
            'mycelium', 'Bread\sWallet', 'trezor', 'ledger\snano', 'Silk\sRoad', 'darknet\smarket', 
            'dogecoin', 'ASIC\sMiner', 'Central\sLedger', 'Hashrate', 'ICO',         
           ]

In [9]:
'|'.join(hashtags)

'bitcoin|blockchain|btc|cryptocurrency|market\\svalue|crypto|ethereum|fintech|coin|doge|ethereum|ripple|litecoin|cardano|monero|TRON|zcash|jaxx|copay|bitpay|exodus|mycelium|Bread\\sWallet|trezor|ledger\\snano|Silk\\sRoad|darknet\\smarket|dogecoin|ASIC\\sMiner|Central\\sLedger|Hashrate|ICO'

In [10]:
bitcoin_pathes = []
for path in (unprocessed_tweets):
    with open(path, 'r') as fp:
        data = json.load(fp)
        if re.search('|'.join(hashtags), json.dumps(data)):
            bitcoin_pathes.append(path)
        #print(data['entities']['hashtags'])


In [11]:
len(bitcoin_pathes)

457